In [1]:
"""
Esempi presi da: 
http://scikit-learn.org/stable/modules/preprocessing.html
"""
"""

4.3 PREPROCESSING DATA

    The sklearn.preprocessing package provides several common utility functions and transformer classes 
    to change raw feature vectors into a representation that is more suitable for the downstream estimators.

    In general, learning algorithms benefit from standardization of the data set. If some outliers are present in 
    the set, robust scalers or transformers are more appropriate. The behaviors of the different scalers, 
    transformers, and normalizers on a dataset containing marginal outliers is highlighted in Compare the effect
    of different scalers on data with outliers.

"""


"""

4.3.1 STANDARDIZATION, OR MEAN REMOVAL AND VARIANCE SCALING

    Standardization of datasets is a common requirement for many machine learning estimators 
    implemented in scikit-learn; they might behave badly if the individual features do not more or less look 
    like standard normally distributed data: Gaussian with zero mean and unit variance.

    In practice we often ignore the shape of the distribution and just transform the data to center
    it by removing the mean value of each feature, then scale it by dividing non-constant features 
    by their standard deviation.

    For instance, many elements used in the objective function of a learning algorithm (such as the RBF
    kernel of Support Vector Machines or the l1 and l2 regularizers of linear models) assume that all 
    features are centered around zero and have variance in the same order. If a feature has a variance 
    that is orders of magnitude larger than others, it might dominate the objective function and make 
    the estimator unable to learn from other features correctly as expected.

    The function scale provides a quick and easy way to perform this operation on a single array-like dataset:

"""
from sklearn import preprocessing
import numpy as np

X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

X_scaled = preprocessing.scale(X_train)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [2]:
# Scaled data has zero mean and unit variance:
X_scaled.mean(axis=0)

array([ 0.,  0.,  0.])

In [3]:
X_scaled.std(axis=0)

array([ 1.,  1.,  1.])

In [4]:
"""
    The preprocessing module further provides a utility class StandardScaler that implements 
    the Transformer API to compute the mean and standard deviation on a training set so as to be able 
    to later reapply the same transformation on the testing set. This class is hence suitable for use 
    in the early steps of a sklearn.pipeline.Pipeline:
"""
scaler = preprocessing.StandardScaler().fit(X_train)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [5]:
scaler.mean_

array([ 1.        ,  0.        ,  0.33333333])

In [6]:
scaler.scale_

array([ 0.81649658,  0.81649658,  1.24721913])

In [7]:
scaler.transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [8]:
"""
    The scaler instance can then be used on new data to transform it the same way it did on the 
    training set:
"""
X_test = [[-1.,1,0.]]
scaler.transform(X_test)

array([[-2.44948974,  1.22474487, -0.26726124]])

In [9]:
"""
    It is possible to disable either centering or scaling by either passing with_mean=False or
    with_std=False to the constructor of StandardScaler.
"""
scaler = preprocessing.StandardScaler(X_train, with_mean= False, with_std= False)

In [10]:
"""
4.3.1.1 SCALING FEATURES TO A RANGE

    An alternative standardization is scaling features to lie between a given minimum and maximum value, 
    often between zero and one, or so that the maximum absolute value of each feature is scaled to unit size. 
    This can be achieved using MinMaxScaler or MaxAbsScaler, respectively.

    The motivation to use this scaling include robustness to very small standard deviations of features
    and preserving zero entries in sparse data.

    Here is an example to scale a toy data matrix to the [0, 1] range:

"""
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[ 0.5       ,  0.        ,  1.        ],
       [ 1.        ,  0.5       ,  0.33333333],
       [ 0.        ,  1.        ,  0.        ]])

In [11]:
"""
    The same instance of the transformer can then be applied to some new test data unseen during the 
    fit call: the same scaling and shifting operations will be applied to be consistent with 
    the transformation performed on the train data:
"""
X_test = np.array([[-3., -1., 4.]])
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax

array([[-1.5       ,  0.        ,  1.66666667]])

In [12]:
"""
    It is possible to introspect the scaler attributes to find about the exact nature of 
    the transformation learned on the training data:
"""
min_max_scaler.scale_

array([ 0.5       ,  0.5       ,  0.33333333])

In [13]:
min_max_scaler.min_

array([ 0.        ,  0.5       ,  0.33333333])

In [14]:
"""
    If MinMaxScaler is given an explicit feature_range=(min, max) the full formula is:
"""
X = X_train
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
min = 1
max = 2
X_scaled = X_std * (max - min) + min

In [15]:
"""
    MaxAbsScaler works in a very similar fashion, but scales in a way that the training data lies within 
    the range [-1, 1] by dividing through the largest maximum value in each feature. It is meant for data 
    that is already centered at zero or sparse data.

    Here is how to use the toy data from the previous example with this scaler:
"""
max_abs_scaled = preprocessing.MaxAbsScaler()
X_train_maxabs = max_abs_scaled.fit_transform(X_train)
X_train_maxabs

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

In [16]:
X_test_maxabs = max_abs_scaled.transform(X_test)
X_test_maxabs

array([[-1.5, -1. ,  2. ]])

In [17]:
max_abs_scaled.scale_

array([ 2.,  1.,  2.])

In [18]:
"""
    As with scale, the module further provides convenience functions minmax_scale and maxabs_scale if 
    ou don’t want to create an object.
"""
"""
4.3.1.2 SCALING SPARSE DATA

    Centering sparse data would destroy the sparseness structure in the data, and thus rarely is a
    sensible thing to do. However, it can make sense to scale sparse inputs, especially if features 
    are on different scales.

    MaxAbsScaler and maxabs_scale were specifically designed for scaling sparse data, and are the 
    recommended way to go about this. However, scale and StandardScaler can accept scipy.sparse
    matrices as input, as long as with_mean=False is explicitly passed to the constructor. Otherwise a
    ValueError will be raised as silently centering would break the sparsity and would often crash the 
    execution by allocating excessive amounts of memory unintentionally. RobustScaler cannot be fitted to 
    sparse inputs, but you can use the transform method on sparse inputs.
    
    Note that the scalers accept both Compressed Sparse Rows and Compressed Sparse Columns 
    format (see scipy.sparse.csr_matrix and scipy.sparse.csc_matrix). Any other sparse input will be 
    converted to the Compressed Sparse Rows representation. To avoid unnecessary memory copies, it
    is recommended to choose the CSR or CSC representation upstream.
    
    Finally, if the centered data is expected to be small enough, explicitly converting the input 
    to an array using the toarray method of sparse matrices is another option.
    
"""

"""
4.3.1.3 SCALING DATA WITH OUTLIERS
    
    If your data contains many outliers, scaling using the mean and variance of the data is likely 
    to not work very well. In these cases, you can use robust_scale and RobustScaler as drop-in 
    replacements instead. They use more robust estimates for the center and range of your data.
    
    REFERENCE: Further discussion on the importance of centering and scaling data is available on this FAQ:
    Should I normalize/standardize/rescale the data?
    
    Scaling vs Whitening
    
    It is sometimes not enough to center and scale the features independently, since a downstream 
    model can further make some assumption on the linear independence of the features.

    To address this issue you can use sklearn.decomposition.PCA or sklearn.decomposition.RandomizedPCA 
    with whiten=True to further remove the linear correlation across features.
    
    Scaling target variables in regression
    
    scale and StandardScaler work out-of-the-box with 1d arrays. This is very useful for scaling 
    the target / response variables used for regression.
    
4.3.2 NON-LINEAR TRANSFORMATION

    Like scalers, QuantileTransformer puts each feature into the same range or distribution. 
    However, by performing a rank transformation, it smooths out unusual distributions and is less 
    influenced by outliers than scaling methods. It does, however, distort correlations and 
    distances within and across features.
    
    QuantileTransformer and quantile_transform provide a non-parametric transformation based on 
    the quantile function to map the data to a uniform distribution with values between 0 and 1:
"""
from sklearn.datasets import load_iris
from sklearn.model_selection import  train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
quantile_transformer = preprocessing.QuantileTransformer(random_state = 0)
X_train_trans = quantile_transformer.fit_transform(X_train)
X_test_trans = quantile_transformer.transform(X_test)
np.percentile(X_train[:,0], [0, 25, 50, 75, 100])

array([ 4.3,  5.1,  5.8,  6.5,  7.9])

In [19]:
"""
    This feature corresponds to the sepal length in cm. Once the quantile transformation applied, 
    those landmarks approach closely the percentiles previously defined:
"""
np.percentile(X_train_trans[:,0], [0, 25, 50, 75, 100])

array([  9.99999998e-08,   2.38738739e-01,   5.09009009e-01,
         7.43243243e-01,   9.99999900e-01])

In [20]:
"""
    This can be confirmed on a independent testing set with similar remarks:
"""
np.percentile(X_test[:,0], [0, 25, 50, 75, 100])

array([ 4.4  ,  5.125,  5.75 ,  6.175,  7.3  ])

In [21]:
np.percentile(X_test_trans[:, 0], [0, 25, 50, 75, 100])

array([ 0.01351351,  0.25012513,  0.47972973,  0.6021021 ,  0.94144144])

In [22]:
"""
    It is also possible to map the transformed data to a normal distribution by setting 
    output_distribution='normal':
"""
quantile_transformer = preprocessing.QuantileTransformer(
    output_distribution = 'normal', random_state = 0)
X_trans = quantile_transformer.fit_transform(X)
quantile_transformer.quantiles_
"""
    Thus the median of the input becomes the mean of the output, centered at 0. The normal output 
    is clipped so that the input’s minimum and maximum — corresponding to the 1e-7 and 1 - 1e-7 
    quantiles respectively — do not become infinite under the transformation.
"""

'\n    Thus the median of the input becomes the mean of the output, centered at 0. The normal output \n    is clipped so that the input’s minimum and maximum — corresponding to the 1e-7 and 1 - 1e-7 \n    quantiles respectively — do not become infinite under the transformation.\n'

In [23]:
"""
4.3.3 NORMALIZATION

    Normalization is the process of scaling individual samples to have unit norm. 
    This process can be useful if you plan to use a quadratic form such as the dot-product 
    or any other kernel to quantify the similarity of any pair of samples.

    This assumption is the base of the Vector Space Model often used in text classification 
    and clustering contexts.

    The function normalize provides a quick and easy way to perform this operation on a single array-like 
    dataset, either using the l1 or l2 norms:

"""
X_normalized = preprocessing.normalize(X, norm='l2')
X_normalized

array([[ 0.80377277,  0.55160877,  0.22064351,  0.0315205 ],
       [ 0.82813287,  0.50702013,  0.23660939,  0.03380134],
       [ 0.80533308,  0.54831188,  0.2227517 ,  0.03426949],
       [ 0.80003025,  0.53915082,  0.26087943,  0.03478392],
       [ 0.790965  ,  0.5694948 ,  0.2214702 ,  0.0316386 ],
       [ 0.78417499,  0.5663486 ,  0.2468699 ,  0.05808704],
       [ 0.78010936,  0.57660257,  0.23742459,  0.0508767 ],
       [ 0.80218492,  0.54548574,  0.24065548,  0.0320874 ],
       [ 0.80642366,  0.5315065 ,  0.25658935,  0.03665562],
       [ 0.81803119,  0.51752994,  0.25041771,  0.01669451],
       [ 0.80373519,  0.55070744,  0.22325977,  0.02976797],
       [ 0.786991  ,  0.55745196,  0.26233033,  0.03279129],
       [ 0.82307218,  0.51442011,  0.24006272,  0.01714734],
       [ 0.8025126 ,  0.55989251,  0.20529392,  0.01866308],
       [ 0.81120865,  0.55945424,  0.16783627,  0.02797271],
       [ 0.77381111,  0.59732787,  0.2036345 ,  0.05430253],
       [ 0.79428944,  0.

In [24]:
"""
    The preprocessing module further provides a utility class Normalizer that implements 
    the same operation using the Transformer API (even though the fit method is useless in this case: 
    the class is stateless as this operation treats samples independently).

    This class is hence suitable for use in the early steps of a sklearn.pipeline.Pipeline:
"""
normalizer = preprocessing.Normalizer().fit(X)
normalizer

Normalizer(copy=True, norm='l2')

In [25]:
"""
    The normalizer instance can then be used on sample vectors as any transformer:
"""
normalizer.transform(X)

array([[ 0.80377277,  0.55160877,  0.22064351,  0.0315205 ],
       [ 0.82813287,  0.50702013,  0.23660939,  0.03380134],
       [ 0.80533308,  0.54831188,  0.2227517 ,  0.03426949],
       [ 0.80003025,  0.53915082,  0.26087943,  0.03478392],
       [ 0.790965  ,  0.5694948 ,  0.2214702 ,  0.0316386 ],
       [ 0.78417499,  0.5663486 ,  0.2468699 ,  0.05808704],
       [ 0.78010936,  0.57660257,  0.23742459,  0.0508767 ],
       [ 0.80218492,  0.54548574,  0.24065548,  0.0320874 ],
       [ 0.80642366,  0.5315065 ,  0.25658935,  0.03665562],
       [ 0.81803119,  0.51752994,  0.25041771,  0.01669451],
       [ 0.80373519,  0.55070744,  0.22325977,  0.02976797],
       [ 0.786991  ,  0.55745196,  0.26233033,  0.03279129],
       [ 0.82307218,  0.51442011,  0.24006272,  0.01714734],
       [ 0.8025126 ,  0.55989251,  0.20529392,  0.01866308],
       [ 0.81120865,  0.55945424,  0.16783627,  0.02797271],
       [ 0.77381111,  0.59732787,  0.2036345 ,  0.05430253],
       [ 0.79428944,  0.

In [26]:
normalizer.transform([[-1., 1., 0]])

array([[-0.70710678,  0.70710678,  0.        ]])

In [27]:
"""
    Sparse input:
    
    normalize and Normalizer accept both dense array-like and sparse matrices from scipy.sparse as input.
    
    For sparse input the data is converted to the Compressed Sparse Rows representation 
    (see scipy.sparse.csr_matrix) before being fed to efficient Cython routines. To avoid unnecessary 
    memory copies, it is recommended to choose the CSR representation upstream.
"""
"""
4.3.4 BINARIZATION

4.3.4.1 FEATURES BINARIZATION
    
    Feature binarization is the process of thresholding numerical features to get boolean values. 
    This can be useful for downstream probabilistic estimators that make assumption that the input data 
    is distributed according to a multi-variate Bernoulli distribution. For instance, this is the case for 
    the sklearn.neural_network.BernoulliRBM.
    
    It is also common among the text processing community to use binary feature values (probably to 
    simplify the probabilistic reasoning) even if normalized counts (a.k.a. term frequencies) or 
    TF-IDF valued features often perform slightly better in practice.

    As for the Normalizer, the utility class Binarizer is meant to be used in the early stages of 
    sklearn.pipeline.Pipeline. The fit method does nothing as each sample is treated independently of others:

"""
binarizer = preprocessing.Binarizer().fit(X)
binarizer

Binarizer(copy=True, threshold=0.0)

In [28]:
binarizer.transform(X)

array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  

In [29]:
"""
    It is possible to adjust the threshold of the binarizer:
"""
binarizer = preprocessing.Binarizer(threshold=1.1)
binarizer.transform(X)

array([[ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 1.,  

In [30]:
"""
    As for the StandardScaler and Normalizer classes, the preprocessing module provides a companion 
    function binarize to be used when the transformer API is not necessary.
"""
"""
    SPARSE INPUT
    
    binarize and Binarizer accept both dense array-like and sparse matrices from scipy.sparse as input.
    
    For sparse input the data is converted to the Compressed Sparse Rows representation 
    (see scipy.sparse.csr_matrix). To avoid unnecessary memory copies, it is recommended to choose the 
    CSR representation upstream.
"""
print('')

### 4.3.5 ENCODING CATEGORICAL FEATURES

In [31]:
"""
    Often features are not given as continuous values but categorical. For example a person could 
    have features ["male", "female"], ["from Europe", "from US", "from Asia"], ["uses Firefox", "uses Chrome", 
    "uses Safari", "uses Internet Explorer"]. Such features can be efficiently coded as integers, 
    for instance ["male", "from US", "uses Internet Explorer"] could be expressed as [0, 1, 3] while 
    ["female", "from Asia", "uses Chrome"] would be [1, 2, 1].

    Such integer representation can not be used directly with scikit-learn estimators, as these expect 
    continuous input, and would interpret the categories as being ordered, which is often not desired 
    (i.e. the set of browsers was ordered arbitrarily).

    One possibility to convert categorical features to features that can be used with scikit-learn 
    estimators is to use a one-of-K or one-hot encoding, which is implemented in OneHotEncoder. 
    This estimator transforms each categorical feature with m possible values into m binary features, 
    with only one active.

"""
enc = preprocessing.OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])  

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [32]:
enc.transform([[0, 1, 3]]).toarray()

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

In [33]:
"""
    By default, how many values each feature can take is inferred automatically from the dataset. 
    It is possible to specify this explicitly using the parameter n_values. There are two genders, 
    three possible continents and four web browsers in our dataset. Then we fit the estimator, and 
    transform a data point. In the result, the first two numbers encode the gender, the next set of three
    numbers the continent and the last four the web browser.

    Note that, if there is a possibility that the training data might have missing categorical features, 
    one has to explicitly set n_values. For example,
"""
enc = preprocessing.OneHotEncoder(n_values=[2, 3, 4])
# Nore that here are missing categorical values for the 2nd and 3rd features
enc.fit([[1, 2, 3],[0, 2, 0]])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values=[2, 3, 4], sparse=True)

In [34]:
enc.transform([[1, 0, 0]]).toarray()

array([[ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.]])

In [35]:
"""
    See Loading features from dicts for categorical features that are represented as a dict, not as integers.
"""
"""
4.3.6 IMPUTATION OF MISSING VALUES

    For various reasons, many real world datasets contain missing values, often encoded as blanks, 
    NaNs or other placeholders. Such datasets however are incompatible with scikit-learn estimators 
    which assume that all values in an array are numerical, and that all have and hold meaning. A basic 
    strategy to use incomplete datasets is to discard entire rows and/or columns containing missing values. 
    However, this comes at the price of losing data which may be valuable (even though incomplete). A better 
    strategy is to impute the missing values, i.e., to infer them from the known part of the data.
    
    The Imputer class provides basic strategies for imputing missing values, either using the mean, 
    the median or the most frequent value of the row or column in which the missing values are located. 
    This class also allows for different missing values encodings.
    
    The following snippet demonstrates how to replace missing values, encoded as np.nan, using the mean 
    value of the columns (axis 0) that contain the missing values:
"""
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imp.fit([[1, 2], [np.nan, 3], [7, 6]])

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [36]:
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))

[[ 4.          2.        ]
 [ 6.          3.66666667]
 [ 7.          6.        ]]


In [37]:
"""
    The Imputer class also supports sparse matrices:
"""
import scipy.sparse as sp
X = sp.csc_matrix([[1, 2], [0, 3], [7, 6]])
imp = Imputer(missing_values=0, strategy='mean', axis=0)
imp.fit(X)

Imputer(axis=0, copy=True, missing_values=0, strategy='mean', verbose=0)

In [38]:
X_test = sp.csc_matrix([[0, 2], [6, 0], [7, 6]])
print(imp.transform(X_test))

[[ 4.          2.        ]
 [ 6.          3.66666667]
 [ 7.          6.        ]]


In [39]:
"""
    Note that, here, missing values are encoded by 0 and are thus implicitly stored in the matrix. 
    This format is thus suitable when there are many more missing values than observed values.
    
    Imputer can be used in a Pipeline as a way to build a composite estimator that supports imputation. 
    See Imputing missing values before building an estimator.
"""
"""

5.3.7 GENERATIONG POLYNOMIAL FEATURES
    Often it’s useful to add complexity to the model by considering nonlinear features of the input data.
    A simple and common method to use is polynomial features, which can get features’ high-order and 
    interaction terms. It is implemented in PolynomialFeatures:
"""
from sklearn.preprocessing import PolynomialFeatures
X = np.arange(6).reshape(3,2)
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [40]:
poly = PolynomialFeatures(2)
poly.fit_transform(X)

array([[  1.,   0.,   1.,   0.,   0.,   1.],
       [  1.,   2.,   3.,   4.,   6.,   9.],
       [  1.,   4.,   5.,  16.,  20.,  25.]])

In [41]:
"""
    The features of X have been transformed from (X_1, X_2) to (1, X_1, X_2, X_1^2, X_1X_2, X_2^2).

    In some cases, only interaction terms among features are required, and it can be gotten with the 
    setting interaction_only=True:
"""
X = np.arange(9).reshape(3, 3)
X

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [42]:
poly = PolynomialFeatures(degree=3, interaction_only=True)
poly.fit_transform(X)

array([[   1.,    0.,    1.,    2.,    0.,    0.,    2.,    0.],
       [   1.,    3.,    4.,    5.,   12.,   15.,   20.,   60.],
       [   1.,    6.,    7.,    8.,   42.,   48.,   56.,  336.]])

In [43]:
"""
    The features of X have been transformed from (X_1, X_2, X_3) to 
    (1, X_1, X_2, X_3, X_1X_2, X_1X_3, X_2X_3, X_1X_2X_3).

    Note that polynomial features are used implicitly in kernel methods (e.g., sklearn.svm.SVC, 
    sklearn.decomposition.KernelPCA) when using polynomial Kernel functions.

    See Polynomial interpolation for Ridge regression using created polynomial features.
"""
"""
4.3.8 CUSTOM TRANSFORMERS

    Often, you will want to convert an existing Python function into a transformer to assist 
    in data cleaning or processing. You can implement a transformer from an arbitrary function
    with FunctionTransformer. For example, to build a transformer that applies a log transformation
    in a pipeline, do:
"""
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p)
X = np.array([[0, 1], [2, 3]])
transformer.transform(X)

array([[ 0.        ,  0.69314718],
       [ 1.09861229,  1.38629436]])

In [44]:
"""
    For a full code example that demonstrates using a FunctionTransformer to do custom feature selection,
    see Using FunctionTransformer to select columns
"""

'\n    For a full code example that demonstrates using a FunctionTransformer to do custom feature selection,\n    see Using FunctionTransformer to select columns\n'